### Inicio

In [36]:
from datetime import date
import os
os.environ["AWS_PROFILE"] = "men"
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [37]:
params = {
    "class_name":"Inteligencia Artificial",
    "target_audience_edu_level":"Preescolar",
    "grade":"no aplica"
}

### Retriever

In [38]:
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever

In [39]:
kb_context_id = "BC3JS4SRQG"
kb_result_id = "MQPBML7IQ2"

# Base knowledge
retriever_context = AmazonKnowledgeBasesRetriever(
    knowledge_base_id = kb_context_id,
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            'overrideSearchType': "SEMANTIC" # optional
        }
    },
)

retriever_result = AmazonKnowledgeBasesRetriever(
    knowledge_base_id = kb_result_id,
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            'overrideSearchType': "SEMANTIC" # optional
        }
    },
)

In [40]:
def get_retriver_docs(docs):    
    context = "\n".join([doc.page_content for doc in docs])  # Iteramos de forma síncrona si 'docs' es una lista
    return context

In [41]:
async def get_retriver_context(params):
    docs = await retriever_context.ainvoke(params)
    return get_retriver_docs(docs)

In [42]:
async def get_retriver_result(params):
    docs = await retriever_result.ainvoke(params)
    return get_retriver_docs(docs)

### Prompts

In [43]:
from langchain.prompts import PromptTemplate

In [44]:
def get_relevant_info():
    with open("get_relevant_info.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
        input_variables=["class_name"],
        template=prompt_content,
    )



In [45]:
def get_base_prompt():
    with open("get_base_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()

    return PromptTemplate(
        input_variables=["target_audience_edu_level", "class_name", "grade", "date", "context", "learning_result"],
        template=prompt_content
    )

In [46]:
def get_key_concepts():
    with open("get_key_concepts.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["target_audience_edu_level", "class_name", "grade", "context"],
            template=prompt_content
        )

In [47]:
def get_activities_prompt():
    with open("get_activities_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["target_audience_edu_level", "class_name", "grade", "context", "dba"],
            template=prompt_content
        )

In [48]:
def get_exam_prompt():
    with open("get_exam_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["prompt"],
            template=prompt_content
        )

In [49]:
def get_coherence_prompt():
    with open("get_coherence_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
        input_variables=["result_1", "result_2", "result_3", "result_4", "target_audience_edu_level"],
        template=prompt_content,
    )

In [50]:
def get_preschool_able_prompt():
    with open("get_preschool_able_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["result_1", "result_2", "result_3", "result_4"],
            template=prompt_content
        )

### Definir el LLM


In [51]:
from langchain_aws import BedrockLLM
from langchain_aws import ChatBedrock, ChatBedrockConverse


bedrock_model = "anthropic.claude-3-sonnet-20240229-v1:0"


llm_instance = ChatBedrockConverse(
    model=bedrock_model,
    temperature=0.02,
    max_tokens=None
)


### Objetos necesarios para definir la cadena y poder invocarla

In [52]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [53]:

def invoke_chain(prompt_template, params):
    
    # Create the LLM chain with prompt and parser
    chain = prompt_template | llm_instance | parser
    return chain.invoke(params)

def validate_input(class_name):

    prompt_content = """ consideras que se puede crear una clase formativa con este tema {class_name}, de no tener suficiente informacion o de que el tema no tenga que ver con aspectos educativos dame un NO por respuesta, de lo contrario dame un SI, delimitate a responder con un SI o un NO """

    appropiate_prompt = PromptTemplate(
            input_variables=["class_name"],
            template=prompt_content
        )

    return invoke_chain(appropiate_prompt, {"class_name": class_name})




In [54]:
relevance = get_relevant_info() | llm_instance | parser
relevance_params = {
    "class_name": params["class_name"]
}

relevante = relevance.invoke(relevance_params)



### Proceso Parte 1 :  Resultados de Aprendizaje

In [55]:
if  validate_input(params["class_name"]) == "NO" or relevante == 'NO':

  
  print("El tema que estás buscando no corresponde a un tema formativo, o actualmente no contamos con suficiente información para generar una clase sobre este asunto.")

else:
  prompt_template = get_base_prompt()
  classes = params["class_name"]
  context_query = f"{params['class_name']}, para grados {params['grade']}"
  context = get_retriver_context(context_query)

  dba_query = f"De los derechos basicos de aprendizaje dame la metodologia de enseñanza para la tematica de ({classes}) dirigida a estudiantes del grado {params['grade']}"
  dba = get_retriver_result(dba_query)

  learning_result_query = f"Dame información de los Estándares Básicos de Competencias y Orientaciones pedagógicas para la clase ({classes}) dirigida a estudiantes del grado {params['grade']}"
  learning_result = get_retriver_result(learning_result_query)

  chain = prompt_template | llm_instance | parser

  new_params = {
            "class_name": params["class_name"],
                "target_audience_edu_level":params["target_audience_edu_level"],
                "grade": params["grade"],
                "context": context,
                "dba": dba,
                "learning_result": learning_result,
                "date":  date.today()
            } 
  
  result_1 = chain.invoke(new_params)




C:\Users\Mateo Grisales\AppData\Local\Temp\ipykernel_10236\1211416754.py:20: RuntimeWarning: coroutine 'get_retriver_result' was never awaited
  new_params = {


### Proceso parte 2 : Conceptos clave

In [56]:
key_template = get_key_concepts()

chain_key_concepts = key_template | llm_instance | parser
key_params = {
    "class_name": params["class_name"],
    "target_audience_edu_level":params["target_audience_edu_level"],
    "grade": params["grade"],
    "context": context
}

result_2 = chain_key_concepts.invoke(key_params)

### Proceso parte 3 : Actividades de aprendizaje

In [57]:
activities_template = get_activities_prompt()

chain_activities = activities_template | llm_instance | parser
activities_params = {
    "class_name": params["class_name"],
    "target_audience_edu_level": params["target_audience_edu_level"],
    "grade": params["grade"],
    "context": context,
    "dba": dba
}

result_3 = chain_activities.invoke(activities_params)

C:\Users\Mateo Grisales\AppData\Local\Temp\ipykernel_10236\942689969.py:4: RuntimeWarning: coroutine 'get_retriver_context' was never awaited
  activities_params = {
C:\Users\Mateo Grisales\AppData\Local\Temp\ipykernel_10236\942689969.py:4: RuntimeWarning: coroutine 'get_retriver_result' was never awaited
  activities_params = {


### Proceso parte 4 : Actividad evaluativa

In [58]:
exam_template = get_exam_prompt()
chain_exam = exam_template | llm_instance | parser
examn_params = {
    "prompt": result_1 + '\n\n' + result_2 + '\n\n' + result_3 
}

result_4 = chain_exam.invoke(examn_params)

### Proceso parte 5 : Coherencia

In [59]:
coherence = get_coherence_prompt() | llm_instance | parser
coherence_params = {
    "result_1": result_1,
    "result_2": result_2,
    "result_3": result_3,
    "result_4": result_4,
    "target_audience_edu_level": params["target_audience_edu_level"]
}

result_5 = coherence.invoke(coherence_params)

In [60]:
if params["target_audience_edu_level"] == "Preescolar":
    preschool_able = get_preschool_able_prompt() | llm_instance | parser
    preschool_able_params = {"result_5": result_5}
    result_5 = preschool_able.invoke(preschool_able_params)

In [61]:
print(result_5)

$result_1$
# ¡Explorando las Máquinas Inteligentes!

Fecha de creación: 2025-01-21
Nivel educativo: Preescolar

Resultados de aprendizaje:

1. Identificar máquinas inteligentes en tu hogar y alrededor.
2. Desarrollar curiosidad sobre cómo funcionan las máquinas.
3. Divertirte aprendiendo sobre máquinas inteligentes a través de juegos.
4. Trabajar en equipo y compartir con tus amigos.

$result_2$
## Conceptos clave:

- **Instrucciones**: Pasos que le decimos a una máquina para que haga algo.
- **Aprender**: Cuando las máquinas mejoran con la práctica.
- **Información**: Lo que las máquinas necesitan para aprender.
- **Máquinas inteligentes**: Máquinas que pueden pensar y hacer tareas como las personas.
- **Máquina**: Dispositivo que puede seguir instrucciones.
- **Patrones**: Formas, colores o tamaños similares.
- **Robots**: Máquinas que pueden moverse y hacer tareas.

$result_3$
## Metodologías

- Aprender jugando: Usar juegos divertidos para aprender sobre máquinas inteligentes.
- Ap

In [62]:
def split_results(full_text):
    final_results = {}
    sections = full_text.split("$result_")

    for section in sections:
        if section.strip(): 
            section_number, content = section.split("$", 1)
            key = f"final_result_{section_number.strip()}"
            final_results[key] = content.strip()

    return final_results

results = split_results(result_5)

result_1 = results.get("result_1", "")
result_2 = results.get("result_2", "")
result_3 = results.get("result_3", "")
result_4 = results.get("result_4", "")

# Ouput

In [63]:
print(result_1)
print()
print(result_2)
print()
print(result_3)
print()
print(result_4)

# ¡Explorando la Inteligencia Artificial!

Fecha de creación: 2025-01-20

Nivel educativo: Preescolar

Resultados de aprendizaje:

1. Identificar ejemplos de Inteligencia Artificial en su entorno cotidiano.
2. Reconocer la importancia de la Inteligencia Artificial en diversas áreas.
3. Desarrollar habilidades de pensamiento crítico y resolución de problemas a través de actividades lúdicas relacionadas con la Inteligencia Artificial.
4. Fomentar la curiosidad y el interés por el aprendizaje continuo en el campo de la Inteligencia Artificial.

## Conceptos clave:

- **Algoritmo**: Conjunto de instrucciones o pasos que se siguen para resolver un problema o realizar una tarea.
- **Aprendizaje automático**: Capacidad de las máquinas para aprender y mejorar a partir de la experiencia.
- **Datos**: Información que se utiliza para enseñar a las máquinas inteligentes.
- **Inteligencia artificial**: Máquinas o computadoras que pueden pensar y aprender como los humanos.
- **Máquina**: Dispositivo o sistema que realiza tareas de manera automática.
- **Reconocimiento de patrones**: Capacidad de identificar y clasificar formas o dibujos.
- **Robótica**: Estudio de los robots, máquinas que pueden moverse y realizar tareas.

## Metodologías

- Aprendizaje basado en juegos y actividades divertidas, aprovechando el interés natural de los niños por jugar.
- Enfoque multisensorial, utilizando recursos visuales, auditivos y táctiles para facilitar la comprensión.
- Aprendizaje colaborativo, fomentando el trabajo en equipo y la interacción social.

## Actividades de aprendizaje

1. Introducción a la Inteligencia Artificial a través de cuentos y personajes animados.
2. Exploración de ejemplos cotidianos de IA, como asistentes virtuales o juguetes inteligentes.
3. Actividades de reconocimiento de patrones y categorización de objetos.
4. Juegos de resolución de problemas sencillos utilizando instrucciones paso a paso.
5. Creación de robots sencillos con materiales reciclados y programación básica.

## Actividades prácticas

- Construir robots sencillos con materiales reciclados y programarlos para realizar tareas simples.
- Juegos de reconocimiento de patrones y categorización de objetos.
- Actividades de programación básica utilizando herramientas visuales y bloques de código.

## Actividades reflexivas

- Discusiones grupales sobre cómo la IA nos ayuda en la vida diaria.
- Reflexiones sobre la importancia de ser amables con las máquinas inteligentes.
- Debates sobre cómo podemos usar la IA para hacer cosas buenas.

1. ¿Cuál de las siguientes opciones NO es un ejemplo de Inteligencia Artificial en tu entorno cotidiano?

A) Un asistente virtual como Alexa o Siri
B) Un juguete que puede hablar y moverse
C) Un robot que limpia tu casa
D) Un lápiz de colores

2. ¿Qué es un algoritmo?

A) Un conjunto de instrucciones para resolver un problema
B) Un tipo de máquina inteligente
C) Un programa de computadora
D) Un juguete que puede aprender

3. ¿Cuál de estas actividades NO está relacionada con la Inteligencia Artificial?

A) Jugar videojuegos
B) Reconocer patrones en imágenes
C) Programar un robot
D) Pintar un dibujo

4. ¿Qué es el aprendizaje automático?

A) Un tipo de juguete inteligente
B) La capacidad de las máquinas para aprender por sí mismas
C) Un algoritmo para resolver problemas
D) Un programa de computadora

5. ¿Qué es la robótica?

A) El estudio de los robots
B) Un tipo de juguete
C) Un programa de computadora
D) Un algoritmo

6. ¿Cuál de estas opciones NO es un concepto clave de la Inteligencia Artificial?

A) Algoritmo
B) Aprendizaje automático
C) Datos
D) Pintura

7. ¿Qué son los datos en la Inteligencia Artificial?

A) Un tipo de máquina
B) Un algoritmo
C) Información que se utiliza para enseñar a las máquinas inteligentes
D) Un juguete inteligente

8. ¿Qué es el reconocimiento de patrones?

A) Un tipo de juego
B) La capacidad de identificar y clasificar formas o dibujos
C) Un algoritmo
D) Un programa de computadora